In [1]:
import csv

# initializing the titles and rows list 
fields = [] 
rows = [] 
# raw_questions = []
raw_questions_to_hw = dict()

filename = 'contributions.csv'
# reading csv file 
with open(filename, 'r', encoding="utf8") as csvfile: 
    # creating a csv reader object 
    csvreader = csv.reader(csvfile) 
      
    # extracting field names through first row 
    fields = next(csvreader) 
  
    # extracting each data row one by one 
    for row in csvreader: 
        rows.append(row) 

    curr_question = ""
    curr_text = ""
    curr_topic = ""
    curr_folder = ""
    curr_answer = ""
    curr_number = 0
    curr_follow_up = 0
    endorsed = 0
    
    # This will help us take the most recent version because it always comes first
    already_filled = 0
    for row in rows[:]: 
        # parsing each column of a row 
        for col,field in zip(row, fields): 
            if(field == "Post Number" and curr_number != int(col)):
                curr_folders = curr_folder.split()
                for folder in curr_folders:
                    if folder[-1] == ';':
                            folder = folder[:-1]
                    if(curr_question != "" and curr_answer != ""):
                        if folder in raw_questions_to_hw:
                            raw_questions = raw_questions_to_hw[folder]
                            raw_questions.append(curr_question)
                            raw_questions_to_hw.update({folder : raw_questions})
                        else:
                            raw_questions = []
                            raw_questions.append(curr_question)
                            raw_questions_to_hw.update({curr_folder : raw_questions})
                curr_number = int(col)
                curr_follow_up = 0
                curr_question = ""
                already_filled = 0
                curr_answer = ""
                curr_topic = ""
                curr_folder = ""
                endorsed = 0
                
            elif(field == "Submission HTML Removed"): #the question could also be in subject
                curr_text = col
                
            elif(field == "Subject"): #We want to check if this is actually the question eventaully
                curr_topic = col
                
            elif(field == "Folders"):
                curr_folder = col
                
            elif(field == "Endorsed by Instructor"):
                if(col == "TRUE"):
                    endorsed = 1
                else:
                    endorsed = 0
            elif(field == "Part of Post"):
                if(col == "started_off_question"):
                    if(already_filled == 0):
                        curr_question = curr_text
                    already_filled = 0
                elif(col == "updated_question"):
                    # update question entry to post and follow up numbers
                    if(already_filled == 0):
                        curr_question = curr_text
                        already_filled = 1
                        
                elif(col == "updated_i_answer"):
                    # update last entry to the post and follow up number
                    # what if it was appended? how do we update just the end?
                    if(already_filled == 0):
                        curr_answer = curr_answer + "\n" + curr_text
                        already_filled = 1
                    endorsed = 1
                elif(col == "started_off_i_answer"):
                    if(already_filled == 0):
                        curr_answer = curr_answer + "\n" + curr_text
                    already_filled = 0
                    endorsed = 1
                    
                elif(col == "updated_s_answer"):
                    # update last entry to the post and follow up number
                    # what if it was appended? how do we update just the end?
                    if(already_filled == 0):
                        curr_answer = curr_answer + "\n" + curr_text
                        already_filled = 1
                elif(col == "started_off_s_answer"):
                    # append to answer - if it was previously endorsed and were appending should we keep it endorsed?
                    if(already_filled == 0):
                        curr_answer = curr_answer + "\n" + curr_text
                    already_filled = 0
                    
                elif(col == "followup"):
                    # new entry for the new followup question 
                    curr_folders = curr_folder.split()
                    for folder in curr_folders:
                        if folder[-1] == ';':
                            folder = folder[:-1]
                        if folder in raw_questions_to_hw:
                            raw_questions = raw_questions_to_hw[folder]
                            raw_questions.append(curr_question)
                            raw_questions_to_hw.update({folder : raw_questions})
                        else:
                            raw_questions = []
                            raw_questions.append(curr_question)
                            raw_questions_to_hw.update({folder : raw_questions})
                    curr_question = ""
                    curr_answer = ""
                    curr_folder = ""
                    endorsed = 0
                    already_filled = 0
                        
                    curr_follow_up = curr_follow_up + 1
                    curr_question = curr_text
                elif(col == "reply_to_followup"):
                    curr_answer = curr_answer + "\n" + curr_text
                    
num = 0
# for i in raw_questions_to_hw["hw5"]:
#     print(num)
#     print(i)
#     print('\n')
#     num = num+1

# myDB.close()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

from xml.dom import minidom

folder = "hw6"
print(len(raw_questions_to_hw[folder]))

# parse an xml file by name
mydoc = minidom.parse('Posts.xml')

SEitems = mydoc.getElementsByTagName('row')

SEitems_questions = []
for item in SEitems:
    SEitems_questions.append(item.attributes['Body'].value)
    
SEitems_questions = SEitems_questions[1:6000]

vectorizer = TfidfVectorizer(stop_words='english')
X_orig = vectorizer.fit(SEitems_questions)
X = vectorizer.transform(raw_questions_to_hw[folder])

# Number of clusters per homework
true_k = 50
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=10)
model.fit(X)

# print("\n")
# print("Prediction")



In [66]:
# Combine clusters that have references to each other

Y = vectorizer.transform(["Should we fail for empty queues for delete and contains too?"])
prediction = model.predict(Y)
# print(prediction)

# for x in range(276):
#     if model.labels_[x] == 28:
#         print("Question Number: ") 
#         print(x)
#         print(raw_questions_to_hw[folder][x])

In [72]:
# Create a list of lists for each cluser

final_clusters = {}
for i in range(len(raw_questions_to_hw[folder])):
    label = model.labels_[i]
    final_clusters.setdefault(label, []).append(raw_questions_to_hw[folder][i])
    
print(final_clusters)

{4: ['', 'my code wont compile because I removed the kudos problem on deque.ml becuase it had an error without me touching it; how many points will we lose for our code not compiling\n', "I created a counter to count the number of surrounding pixels given the radius, and I divide the r, g, and b values by this counter outside of my for loops. I just can't figure out why I keep getting an ArithmeticException: / by zero. Please help!\xa0", 'When will HW7 be released?', "Is there a picture of what the code files are supposed to look like on Eclipse? I'm getting a bunch of errors because I don't think they're in the right place", 'Oh, thanks! What difference does that exactly make?', 'Can multiple clients have the same nickname in the server state?', 'Are we allowed to use enhanced for loops for this homework?', 'Should we return a copy of strings for getters\xa0or is returning the instance variable fine?', "I don't understand what that means. While implementing a getter\nShould I\xa0\nret

In [75]:
# Shorten the sentences just for one cluster for now
import random
from nltk.tokenize import word_tokenize
import string

number, items = random.choice(list(final_clusters.items()))

stop_words = ["ourselves", "hers", "between", "yourself", "but", "again", 
                  "there", "about", "once", "during", "out", "very", "having", 
                  "with", "they", "own", "an", "be", "some", "for", "do", "its", 
                  "yours", "such", "into", "of", "most", "itself", "other", "off",
                  "is", "s", "am", "or", "who", "as", "from", "him", "each", "the",
                  "themselves", "until", "below", "are", "we", "these", "your", "his", 
                  "through", "don", "nor", "me", "were", "her", "more", "himself", "this",
                  "down", "should", "our", "their", "while", "above", "both", "up", "to",
                  "ours", "had", "she", "all", "no", "when", "at", "any", "before", "them",
                  "same", "and", "been", "have", "in", "will", "on", "does", "yourselves", 
                  "then", "that", "because", "what", "over", "why", "so", "can", "did", 
                  "not", "now", "under", "he", "you", "herself", "has", "just",
                  "where", "too", "only", "myself", "which", "those", "i", "after",
                  "few", "whom", "t", "being", "if", "theirs", "my", "against", "a",
                  "by", "doing", "it", "how", "further", "was", "here", "than", "ok",
                  "thank", "ah"]

length = len(items)
shortened_questions = []
for i in range(0, length):
    question = items[i]
    tokens = word_tokenize(question)
    tokens = [w.lower() for w in tokens]
    words = [w for w in tokens if (not w in stop_words and w != "" and w != " ")]

    # Get rid of punctuation
    table = str.maketrans('', '', string.punctuation)
    words = [w.translate(table) for w in words]

    final_question = " ".join(words)

    shortened_questions.append(final_question)

#     shortened_questions = [s for s in shortened_questions if s != '']

#     # Printing out average
# totalNumWords = 0
# maxLength = 0
# for s in shortened_questions:
#     numWords = len(word_tokenize(s))
#     if (maxLength < numWords): 
#         maxLength = numWords
#     # print(numWords)
#     if (numWords > 50):
#         print(s)
#         print("\n")
#     totalNumWords += numWords

print(shortened_questions)



['connectionnicknamestest  getting  junit tests found   fix error ', 'manual grading section  says  quality testing  5  grade  mean junit tests expected ', 'add tests testing files create new file ', 'add junit tests  add servermodeltest file ', 'finished writing code change nickname  servermodel updateservermodel method nickname command  m passing nick tests   m passing tests register deregister   im struggling find bug code  ve looked failed tests error always line checks broadcast returned matches expected broadcast  debugging suggestions would greatly appreciated ', 'still dont squares crosses file setup  went run tests  said tests found  add junit 4 libraries ', ' s probably  dont see faq talks setting source folders nt setup  ca nt path building stuff  right click source test  option creating new source folder nothing  run tests  runs tests hw 6 fail  even though passed ran submission  ', 'yes understand  nt two tests run submissionconncetionnicknamestest  test run  mean program 

In [76]:
from bert_serving.client import BertClient 

# Start BERT client for mrpc bert-serving-start -model_dir /Users/maralevy/Downloads/uncased_L-12_H-768_A-12 -tuned_model_dir /Users/maralevy/Documents/Piazza-SeniorDesign/bert/mrpc_output -num_worker=4 -max_seq_len=50
# Start BERT client for qqa checkpoint bert-serving-start -model_dir /Users/maralevy/Downloads/uncased_L-12_H-768_A-12 -tuned_model_dir /Users/maralevy/Documents/Piazza-SeniorDesign/bert/qqp_output -num_worker=4 -max_seq_len=50 -ckpt_name bert_model.ckpt-1000
bc = BertClient(check_version=False)  
doc_vecs = bc.encode(shortened_questions)

In [77]:
# This cell allows you to type a question and find semantically similar questions using BURT, eventually
# we should run this for every question in raw_questions

import np
print(doc_vecs)
topk = len(shortened_questions)
final_scores = {}
for question in shortened_questions:
    query_vec = bc.encode([question])
    # compute simple dot product as score
    score = np.sum(query_vec * doc_vecs, axis=1)/np.linalg.norm(doc_vecs, axis=1)
    topk_idx = np.argsort(score)[::-1][:topk]
#     print("Question: ")
#     print(question)
    for idx in topk_idx:
        if not idx == topk_idx[0]:
            final_scores.setdefault(items[topk_idx[0]], []).append((score[idx], items[idx]))

print(final_scores)

[[-1.1728452  -0.0315334   0.3136945  ... -0.06217764 -0.4631861
   0.7519805 ]
 [-0.6832422   0.24118419 -0.00305382 ... -0.37899604 -0.39198294
   0.26979482]
 [-0.4066884   0.2946769   0.25936097 ... -0.43329775 -0.74279225
  -0.13673964]
 ...
 [-1.2626799   0.3872425   0.30454305 ...  0.26641056 -0.73402435
   0.34830773]
 [-0.8956507   0.6663191   0.48223022 ...  0.08717323 -0.6680188
   0.49548435]
 [-0.78980607  0.26755324  0.19435745 ...  0.13773821 -0.77142704
   0.37776646]]
{'For ConnectionNicknamesTest, I am getting a "No JUnit tests found." How do I fix this error?': [(19.842464, "Yes I understand. But why didn't two of the tests run in SubmissionConncetionNicknamesTest? When a test does not run, does that mean the program crashed?"), (19.74009, "I just finished writing code to change nickname (in both serverModel\xa0and the updateServerModel method of the Nickname Command) but I'm not passing any of the Nick tests. (I'm passing all tests for register and deregister). I'm\

In [78]:
for key in final_scores:
    if key == final_scores[final_scores[key][0][1]][0] or key == final_scores[final_scores[key][0][1]][1][1] or key == final_scores[final_scores[key][0][1]][2][1]:
        print("Match: \n%s, \n%s\n\n" % (key, final_scores[key][0][1]))
#     else:
#         print("No Match: \n%s, \n%s\n\n" % (key, final_scores[key][0][1]))

Match: 
I still dont have the squares with the crosses through them in my file setup, and when I went to run my tests, it said that no tests were found. I did add JUnit 4 to the libraries., 
Ok--that's probably it, but I dont see where on the FAQ it talks about setting source folders and that isn't in the setup... I can't do any of that path building stuff, and when I right click on source or test, I only have the option of creating a new source folder which does nothing. When I run the tests, it runs the tests from hw 6 and they all fail (even though they passed when I ran them before submission).


Match: 
When writing JUnit tests, can we just test different command scenarios, which indirectly tests the ServerModel class methods? 

If not, do we have to also test the methods within ServerModel individually (e.g. whether it returns null when it's supposed to)?, 
So I know we have to provide tests for grading for ServerModel, but do we need to provide tests for everything in ServerMode